## Installs

In [ ]:
#Install packages to apply SVD and Bayesian Factorization Machine
!pip install surprise

!pip install myfm

## Imports

In [1]:
# To store the data
import pandas as pd

# To do linear algebra
import numpy as np

# To apply SVD
import surprise

# To apply Factorization Machines
import myfm
from myfm import RelationBlock
from myfm.utils.encoders import(DataFrameEncoder,MultipleValuesToSparseEncoder,CategoryValueToSparseEncoder)

# To do train-test split for evaluation
from sklearn.model_selection import train_test_split

## Load Data

In [2]:
#Function to Extract users, movies and ratings from raw data
def extract_users_items_predictions(data_pd):
    users, movies = \
        [np.squeeze(arr) for arr in np.split(data_pd.Id.str.extract('r(\d+)_c(\d+)').values.astype(int) - 1, 2, axis=-1)]
    predictions = data_pd.Prediction.values
    return users, movies, predictions

#Load raw data
data_pd = pd.read_csv('../data/data_train.csv')

#Do Train-Test Split
train, val = train_test_split(data_pd, test_size=0.1,random_state=42)

#Extract users, movies and ratings from raw data
total_users, total_movies, total_pred = extract_users_items_predictions(data_pd)
train_users, train_movies, train_pred = extract_users_items_predictions(train)
val_users, val_movies, val_pred = extract_users_items_predictions(val)

#Store total data
ratings_dict_total = {'userID': total_users,'movieID': total_movies,'rating': total_pred}
df_total = pd.DataFrame(ratings_dict_total)

#Store train data
ratings_dict_train = {'userID': train_users,'movieID': train_movies,'rating': train_pred}
df_train = pd.DataFrame(ratings_dict_train)

#Store validation data
ratings_dict_test = {'userID': val_users,'movieID': val_movies,'rating': val_pred}
df_test = pd.DataFrame(ratings_dict_test)

In [3]:
df_train

,userID,movieID,rating
0,630,740,4
1,5909,785,2
2,7720,545,4
3,7629,45,3
4,2519,934,5
...,...,...,...
1059251,153,92,5
1059252,8739,213,5
1059253,9122,108,2
1059254,8104,574,5


## Get implicit user/movie Features

In [4]:
'''Code adapted from https://github.com/tohtsky/myFM/blob/main/examples/ml-100k-extended.ipynb'''

#Utility function to convert list to str
def string_agg(int_list) -> str:
    return ','.join([str(y) for y in int_list])

def aggr(int_list):
    return [y for y in int_list]

#Get & Store Implicit Features
user_f = pd.DataFrame(np.arange(10000),columns=['userID'])
user_f['implicit_features'] = df_train.groupby('userID').movieID.agg(string_agg).reindex(user_f.index).fillna('')
user_f['rating_features'] = df_train.groupby('userID').rating.agg(aggr).reindex(user_f.index).fillna('')
user_f['user_metadata'] = user_f[['implicit_features', 'rating_features']].to_dict(orient='records')
movie_f = pd.DataFrame(np.arange(1000),columns=['movieID'])
movie_f['implicit_features'] = df_train.groupby('movieID').userID.agg(string_agg).reindex(movie_f.index).fillna('')
movie_f['rating_features'] = df_train.groupby('movieID').rating.agg(aggr).reindex(movie_f.index).fillna('')
movie_f['movie_metadata'] = movie_f[['implicit_features', 'rating_features']].to_dict(orient='records')

In [5]:
from typing import Iterable

import scipy.sparse as sps
from typing_extensions import Literal



class WeightMultipleValuesToSparseEncoder(CategoryValueToSparseEncoder[str]):
    """The class to N-hot encode a List of items into a sparse matrix representation."""

    def __init__(
        self,
        items: Iterable[str],
        min_freq: int = 1,
        sep: str = ",",
        normalize: bool = True,
        handle_unknown: Literal["create", "ignore", "raise"] = "create",
    ):
        """Construct the encoder by providing a list of strings,
        each of which is a list of strings concatenated by `sep`.
        Parameters
        ----------
        items : Iterable[str]
            Iterable of strings, each of which is a concatenated list of possibly multiple items.
        min_freq : int, optional
            The minimal frequency for an item to be retained in the known items list, by default 1.
        sep: str, optional
            Tells how to separate string back into a list. Defaults to `','`.
        normalize: bool, optional
            If `True`, non-zero entry in the encoded matrix will have `1 / N ** 0.5`,
            where `N` is the number of non-zero entries in that row. Defaults to `True`.
        handle_unknown: Literal["create", "ignore", "raise"], optional
            How to handle previously unseen values during encoding.
            If "create", then there is a single category named "__UNK__" for unknown values,
            ant it is treated as 0th category.
            If "ignore", such an item will be ignored.
            If "raise", a `KeyError` is raised.
            Defaults to "create".
        """
        items_flatten = [
            y for x in items for y in set(x['implicit_features'].split(sep)) if y
        ]  # ignore empty string.
        self.sep = sep
        self.normalize = normalize
        super().__init__(
            items_flatten, min_freq=min_freq, handle_unknown=handle_unknown
        )

    def to_sparse(self, items: Iterable[str]) -> sps.csr_matrix:
        indptr = [0]
        indices = []
        data = []
        n_row = 0
        cursor = 0
        for row in items:
            n_row += 1
            items = row['implicit_features'].split(self.sep)
            indices_local = sorted(
                list(
                    {
                        index
                        for index in [self._get_index(v) for v in items if v]
                        if index is not None
                    }
                )
            )

            if not indices_local:
                indptr.append(cursor)
                continue
            n = len(indices_local)
            #value = 1.0 / (float(n) ** 0.5) if self.normalize else 1.0
            indices.extend(indices_local)
            #data.extend([value] * n)
            weights = row['rating_features']
            total_weights = sum(weights)
            values = []
            for i in range(n):
                values.append(weights[i] / total_weights)
            data.extend(values)
                
            cursor += n
            indptr.append(cursor)
        return sps.csr_matrix(
            (data, indices, indptr),
            shape=(n_row, len(self)),
        )


# Transform Data in myfm Format

In [6]:
#Create User Encoder
user_encoder = DataFrameEncoder().add_column(
    'userID', CategoryValueToSparseEncoder(user_f.userID)
    )

user_encoder.add_column(
        'user_metadata',
        WeightMultipleValuesToSparseEncoder(user_f.user_metadata, normalize=True)
    )


#Create Movie Encoder
movie_encoder = DataFrameEncoder().add_column(
    'movieID', CategoryValueToSparseEncoder(movie_f.movieID)
    )
movie_encoder.add_column(
        'movie_metadata',
        WeightMultipleValuesToSparseEncoder(movie_f.movie_metadata, normalize=True)
    )


def augment_user_id(user_ids):
    return user_encoder.encode_df(
        user_f.reindex(user_ids).reset_index()
    )

def augment_movie_id(movie_ids):
    return movie_encoder.encode_df(
        movie_f.reindex(movie_ids).reset_index()
    )


#Create Train/Test Blocks
train_blocks = []
test_blocks = []
for source, target in [(df_train, train_blocks),(df_test, test_blocks)]:
    unique_users, user_map = np.unique(source.userID, return_inverse=True)
    target.append(
        RelationBlock(user_map, augment_user_id(unique_users))
    )
    unique_movies, movie_map = np.unique(source.movieID, return_inverse=True)
    target.append(
        RelationBlock(movie_map, augment_movie_id(unique_movies))
    )

## Evaluation and Parameter Tuning

In [7]:
# Return root mean square error metric
def RMSE(x: np.ndarray, y: np.ndarray) -> float:
    return np.sqrt(np.average((x - y) ** 2))

# Function to do parameter tuning
def validation(user_encoder,movie_encoder,df_train,train_blocks,test_blocks,val_predictions):
    group_shapes = user_encoder.encoder_shapes + movie_encoder.encoder_shapes

    rank = [8,10,12,14]
    n_iter = [100,200,300,400,500]
    n_samples = [95,195,295,395,495]

    best_rmse = 1
    best_r = 0
    best_n = 0
    best_s = 0

    for r in rank:
        for i in range(len(n_iter)):
            fm = myfm.MyFMRegressor(rank=r,random_seed=1234)
            fm.fit(None, df_train.rating.values, X_rel=train_blocks, 
                   group_shapes=group_shapes,n_iter=n_iter[i], n_kept_samples=n_samples[i])
            test_predictions = fm.predict(None, test_blocks)
            rmse = RMSE(val_predictions,test_predictions)
            if rmse < best_rmse:
                best_rmse = rmse
                best_r = r
                best_n = n_iter[i]
                best_s = n_samples[i]
                print(best_rmse)
                
    return best_rmse,best_r,best_n,best_s

best_rmse,best_r,best_n,best_s = validation(user_encoder,movie_encoder,df_train,train_blocks,test_blocks,val_pred)

alpha = 1.12 w0 = 3.78 : 100%|████████████████| 100/100 [01:05<00:00,  1.53it/s]


0.9744198592190163


alpha = 1.12 w0 = 3.81 : 100%|████████████████| 200/200 [02:03<00:00,  1.61it/s]


0.9738884136608861


alpha = 1.12 w0 = 3.85 : 100%|████████████████| 300/300 [03:04<00:00,  1.63it/s]


...

alpha = 1.13 w0 = 3.78 : 100%|████████████████| 200/200 [01:57<00:00,  1.70it/s]


0.9733395483947272


alpha = 1.13 w0 = 3.81 : 100%|████████████████| 300/300 [03:05<00:00,  1.62it/s]


0.9731564794960696


alpha = 1.16 w0 = 3.93 : 100%|████████████████| 500/500 [08:18<00:00,  1.00it/s]


In [8]:
print("Validation RMSE using model is " + str(best_rmse))

Validation RMSE using model is 0.9731564794960696


# Generate Features & Train the model on Total Data

In [9]:
#Get & Store Implicit Features on total data
user_f = pd.DataFrame(np.arange(10000),columns=['userID'])
user_f['implicit_features'] = df_total.groupby('userID').movieID.agg(string_agg).reindex(user_f.index).fillna('')
user_f['rating_features'] = df_total.groupby('userID').rating.agg(aggr).reindex(user_f.index).fillna('')
user_f['user_metadata'] = user_f[['implicit_features', 'rating_features']].to_dict(orient='records')
movie_f = pd.DataFrame(np.arange(1000),columns=['movieID'])
movie_f['implicit_features'] = df_total.groupby('movieID').userID.agg(string_agg).reindex(movie_f.index).fillna('')
movie_f['rating_features'] = df_total.groupby('movieID').rating.agg(aggr).reindex(movie_f.index).fillna('')
movie_f['movie_metadata'] = movie_f[['implicit_features', 'rating_features']].to_dict(orient='records')



#Create User Encoder for total data
user_encoder = DataFrameEncoder().add_column(
    'userID', CategoryValueToSparseEncoder(user_f.userID)
    )

user_encoder.add_column(
        'user_metadata',
        WeightMultipleValuesToSparseEncoder(user_f.user_metadata, normalize=True)
    )


#Create Movie Encoder
movie_encoder = DataFrameEncoder().add_column(
    'movieID', CategoryValueToSparseEncoder(movie_f.movieID)
    )
movie_encoder.add_column(
        'movie_metadata',
        WeightMultipleValuesToSparseEncoder(movie_f.movie_metadata, normalize=True)
    )


def augment_user_id(user_ids):
    return user_encoder.encode_df(
        user_f.reindex(user_ids).reset_index()
    )

def augment_movie_id(movie_ids):
    return movie_encoder.encode_df(
        movie_f.reindex(movie_ids).reset_index()
    )


#Generate Train Blocks on total data
group_shapes = user_encoder.encoder_shapes + movie_encoder.encoder_shapes

total_train_blocks = []
for source, target in [(df_total, total_train_blocks)]:
    unique_users, user_map = np.unique(source.userID, return_inverse=True)
    target.append(
        RelationBlock(user_map, augment_user_id(unique_users))
    )
    unique_movies, movie_map = np.unique(source.movieID, return_inverse=True)
    target.append(
        RelationBlock(movie_map, augment_movie_id(unique_movies))
    )


#Train the model
fm = myfm.MyFMRegressor(rank=best_r,random_seed=1234)
fm.fit(None, df_total.rating.values, X_rel=total_train_blocks,
    group_shapes=group_shapes,
    n_iter=best_n, n_kept_samples=best_s
)

alpha = 1.13 w0 = 3.78 : 100%|████████████████| 300/300 [03:02<00:00,  1.64it/s]


## Generate predictions for Submission

In [10]:
#Load Submission File
sub_pd = pd.read_csv('../data/sampleSubmission.csv')
sub_users, sub_movies, sub_pred = extract_users_items_predictions(sub_pd)
sub_test_ratings_dict = {'userID': sub_users,'movieID': sub_movies,'rating': sub_pred}
sub_df = pd.DataFrame(sub_test_ratings_dict)

#Generate Submission Test Blocks
sub_test_blocks = []
for source, target in [(sub_df, sub_test_blocks)]:
    unique_users, user_map = np.unique(source.userID, return_inverse=True)
    target.append(
        RelationBlock(user_map, augment_user_id(unique_users))
    )
    unique_movies, movie_map = np.unique(source.movieID, return_inverse=True)
    target.append(
        RelationBlock(movie_map, augment_movie_id(unique_movies))
    )

#Generate Predictions and create submission csv    
predictions = fm.predict(None, sub_test_blocks)
sub_pd['Prediction'] = predictions
sub_pd.set_index("Id", inplace = True)
sub_pd.to_csv("submission_weighted_bayesian_svdpp_flipped.csv")